In [204]:
#####-------------------#####

In [205]:
from libs import *

In [206]:
%run Nearest_Kronicker.ipynb

In [207]:
#####-------------------#####

In [208]:
#(1): get state representation
#(2): apply swap operation
#(2): get individual states
#(3): apply reduce kron prod
#(4): perform matrix multiplication with CNOT
#(5): apply nearest_kron_prod
#(6): store the respective kron_prod_out into qubit indices

In [209]:
class QuantumCircuit:
    def __init__(self, num_of_qubits):
        self.n = num_of_qubits
        self._STATE_CACHE_HASH_MAP = dict()
        self._LINE_INDEX_MAP = dict()
        
    def init(self):
        self._STATE_CACHE_HASH_MAP = {uuid.uuid4().hex : KET_0 for i in range(0, self.n)}
        self._LINE_INDEX_MAP = {j :hashval for j, hashval in enumerate(self._STATE_CACHE_HASH_MAP.keys())}
            

    
    def state_vector(self):
        """
        Returns Global State of all qubits only
        """
        global_state = [state_info for hashval, state_info in self._STATE_CACHE_HASH_MAP.items()]
        return reduce(kron, global_state)
        
            


    def X(self, on_qubits):
        if isinstance(on_qubits, int):
            hashval = self._LINE_INDEX_MAP[on_qubits]
            curr_state = self._STATE_CACHE_HASH_MAP[hashval] 
            x_operated =matmul(X, curr_state)
            self._STATE_CACHE_HASH_MAP[hashval] =x_operated

        if isinstance(on_qubits, list):
            state_list = list()
            for qubit in on_qubits:
                hashval = self._LINE_INDEX_MAP[qubit]
                curr_state = self._STATE_CACHE_HASH_MAP[hashval]
                self._STATE_CACHE_HASH_MAP[hashval] = matmul(X, curr_state)

    def Y(self, on_qubits):
        if isinstance(on_qubits, int):
            hashval = self._LINE_INDEX_MAP[on_qubits]
            curr_state = self._STATE_CACHE_HASH_MAP[hashval] 
            y_operated =matmul(Y, curr_state)
            self._STATE_CACHE_HASH_MAP[hashval] =y_operated

        if isinstance(on_qubits, list):
            state_list = list()
            for qubit in on_qubits:
                hashval = self._LINE_INDEX_MAP[qubit]
                curr_state = self._STATE_CACHE_HASH_MAP[hashval]
                self._STATE_CACHE_HASH_MAP[hashval] = matmul(Y, curr_state)

    def Z(self, on_qubits):
        if isinstance(on_qubits, int):
            hashval = self._LINE_INDEX_MAP[on_qubits]
            curr_state = self._STATE_CACHE_HASH_MAP[hashval] 
            z_operated =matmul(Z, curr_state)
            self._STATE_CACHE_HASH_MAP[hashval] =z_operated

        if isinstance(on_qubits, list):
            state_list = list()
            for qubit in on_qubits:
                hashval = self._LINE_INDEX_MAP[qubit]
                curr_state = self._STATE_CACHE_HASH_MAP[hashval]
                self._STATE_CACHE_HASH_MAP[hashval] = matmul(Z, curr_state)


    def H(self, on_qubits):
        if isinstance(on_qubits, int):
            hashval = self._LINE_INDEX_MAP[on_qubits]
            curr_state = self._STATE_CACHE_HASH_MAP[hashval] 
            h_operated =matmul(H, curr_state)
            self._STATE_CACHE_HASH_MAP[hashval] =h_operated

        if isinstance(on_qubits, list):
            state_list = list()
            for qubit in on_qubits:
                hashval = self._LINE_INDEX_MAP[qubit]
                curr_state = self._STATE_CACHE_HASH_MAP[hashval]
                self._STATE_CACHE_HASH_MAP[hashval] = matmul(H, curr_state)

    def CX(self, source_qubit, target_qubit):
        """
        CX is a Two qubit Universal Gate. It is also called Non-local gate( or :At a distance"from Einstein's(EPR Paradox based - Spooky action 'at distance').
            It is a Unitary Gate.
            #(1): get state representation
            #(2): apply swap operation
            #(2): get individual states
            #(3): apply reduce kron prod
            #(4): perform matrix multiplication with CNOT
            #(5): apply nearest_kron_prod
            #(6): store the respective kron_prod_out into qubit indices
        """
        qubits_gap = source_qubit - target_qubit
        if source_qubit < self.n and target_qubit < self.n and abs(qubits_gap) >=1:
            if qubits_gap<0:
                source_qubit_hash = self._LINE_INDEX_MAP[source_qubit]
                source_qubit_state = self._STATE_CACHE_HASH_MAP[source_qubit_hash]
                target_qubit_hash = self._LINE_INDEX_MAP[target_qubit]
                target_qubit_state = self._STATE_CACHE_HASH_MAP[target_qubit_hash]
                cx_current_state = matmul(CNOT, reduce(kron, [source_qubit_state, target_qubit_state]))
                nkp_state = nearest_kron_prod(cx_current_state)
                if nkp_state['ent']:
                    new_hash_key = uuid.uuid4().hex
                    self._LINE_INDEX_MAP[source_qubit] = new_hash_key
                    self._LINE_INDEX_MAP[source_qubit] = nkp_state['qubit-pair']
                    self._LINE_INDEX_MAP[target_qubit] = new_hash_key                
                    self._LINE_INDEX_MAP[target_qubit] = nkp_state['qubit-pair']

                if not nkp_state['ent']:
                    self._LINE_INDEX_MAP[source_qubit] = nkp_state['qubit-pair'][0]           
                    self._LINE_INDEX_MAP[target_qubit] = nkp_state['qubit-pair'][1]

            if qubits_gap>0:
                source_qubit_hash = self._LINE_INDEX_MAP[source_qubit]
                source_qubit_state = self._STATE_CACHE_HASH_MAP[source_qubit_hash]
                target_qubit_hash = self._LINE_INDEX_MAP[target_qubit]
                target_qubit_state = self._STATE_CACHE_HASH_MAP[target_qubit_hash]
                cx_current_state = matmul(INV_CNOT, reduce(kron, [source_qubit_state, target_qubit_state])
                nkp_state =  nearest_kron_prod(cx_current_state)
                if nkp_state['ent']:
                    new_hash_key = uuid.uuid4().hex
                    self._LINE_INDEX_MAP[source_qubit] = new_hash_key
                    self._LINE_INDEX_MAP[source_qubit] = nkp_state['qubit-pair']
                    self._LINE_INDEX_MAP[target_qubit] = new_hash_key                
                    self._LINE_INDEX_MAP[target_qubit] = nkp_state['qubit-pair']

                if not nkp_state['ent']:
                    self._LINE_INDEX_MAP[source_qubit] = nkp_state['qubit-pair'][0]           
                    self._LINE_INDEX_MAP[target_qubit] = nkp_state['qubit-pair'][1]

        else:
            raise Exception("Exception TBA")



    def CZ(self, source_qubit, target_qubit):
        """
        CX is a Two qubit Universal Gate. It is also called Non-local gate( or :At a distance"from Einstein's(EPR Paradox based - Spooky action 'at distance').
            It is a Unitary Gate.
            #(1): get state representation
            #(2): apply swap operation
            #(2): get individual states
            #(3): apply reduce kron prod
            #(4): perform matrix multiplication with CNOT
            #(5): apply nearest_kron_prod
            #(6): store the respective kron_prod_out into qubit indices
        """
        qubits_gap = source_qubit - target_qubit
        print(qubits_gap)
        if source_qubit < self.n and target_qubit < self.n and abs(qubits_gap) >=1:
            if qubits_gap<0:
                source_qubit_hash = self._LINE_INDEX_MAP[source_qubit]
                source_qubit_state = self._STATE_CACHE_HASH_MAP[source_qubit_hash]
                target_qubit_hash = self._LINE_INDEX_MAP[target_qubit]
                target_qubit_state = self._STATE_CACHE_HASH_MAP[target_qubit_hash]
                cz_current_state = matmul(CZ, reduce(kron, [source_qubit_state, target_qubit_state]))
                nkp_state = nearest_kron_prod(cz_current_state)
                print(f"nkp = {nkp_state}")
                if nkp_state['ent']:
                    new_hash_key = uuid.uuid4().hex
                    self._LINE_INDEX_MAP[source_qubit] = new_hash_key
                    self._LINE_INDEX_MAP[source_qubit] = nkp_state['qubit-pair']
                    self._LINE_INDEX_MAP[target_qubit] = new_hash_key                
                    self._LINE_INDEX_MAP[target_qubit] = nkp_state['qubit-pair']

                if not nkp_state['ent']:
                    self._LINE_INDEX_MAP[source_qubit] = nkp_state['qubit-pair'][0]           
                    self._LINE_INDEX_MAP[target_qubit] = nkp_state['qubit-pair'][1]

            if qubits_gap>0:
                source_qubit_hash = self._LINE_INDEX_MAP[source_qubit]
                source_qubit_state = self._STATE_CACHE_HASH_MAP[source_qubit_hash]
                target_qubit_hash = self._LINE_INDEX_MAP[target_qubit]
                target_qubit_state = self._STATE_CACHE_HASH_MAP[target_qubit_hash]
                print("I am here", source_qubit_state)
                cz_current_state = matmul(INV_CZ, reduce(kron, [source_qubit_state, target_qubit_state]))
                nkp_state = nearest_kron_prod(cz_current_state)
                
                print(f"nkp = {nkp_state}")
                if nkp_state['ent']:
                    new_hash_key = uuid.uuid4().hex
                    self._LINE_INDEX_MAP[source_qubit] = new_hash_key
                    self._LINE_INDEX_MAP[source_qubit] = nkp_state['qubit-pair']
                    self._LINE_INDEX_MAP[target_qubit] = new_hash_key                
                    self._LINE_INDEX_MAP[target_qubit] = nkp_state['qubit-pair']

                if not nkp_state['ent']:
                    self._LINE_INDEX_MAP[source_qubit] = nkp_state['qubit-pair'][0]           
                    self._LINE_INDEX_MAP[target_qubit] = nkp_state['qubit-pair'][1]

        else:
            raise Exception("Exception TBA")



    def CY(self, source_qubit, target_qubit):
        """
        CX is a Two qubit Universal Gate. It is also called Non-local gate( or :At a distance"from Einstein's(EPR Paradox based - Spooky action 'at distance').
            It is a Unitary Gate.
            #(1): get state representation
            #(2): apply swap operation
            #(2): get individual states
            #(3): apply reduce kron prod
            #(4): perform matrix multiplication with CNOT
            #(5): apply nearest_kron_prod
            #(6): store the respective kron_prod_out into qubit indices
        """
        qubits_gap = source_qubit - target_qubit
        print(qubits_gap)
        if source_qubit < self.n and target_qubit < self.n and abs(qubits_gap) >=1:
            if qubits_gap<0:
                source_qubit_hash = self._LINE_INDEX_MAP[source_qubit]
                source_qubit_state = self._STATE_CACHE_HASH_MAP[source_qubit_hash]
                target_qubit_hash = self._LINE_INDEX_MAP[target_qubit]
                target_qubit_state = self._STATE_CACHE_HASH_MAP[target_qubit_hash]
                cy_current_state = matmul(CY, reduce(kron, [source_qubit_state, target_qubit_state]))
                nkp_state = nearest_kron_prod(cy_current_state)
                if nkp_state['ent']:
                    new_hash_key = uuid.uuid4().hex
                    self._LINE_INDEX_MAP[source_qubit] = new_hash_key
                    self._LINE_INDEX_MAP[source_qubit] = nkp_state['qubit-pair']
                    self._LINE_INDEX_MAP[target_qubit] = new_hash_key                
                    self._LINE_INDEX_MAP[target_qubit] = nkp_state['qubit-pair']

                if not nkp_state['ent']:
                    self._LINE_INDEX_MAP[source_qubit] = nkp_state['qubit-pair'][0]           
                    self._LINE_INDEX_MAP[target_qubit] = nkp_state['qubit-pair'][1]

            if qubits_gap>0:
                source_qubit_hash = self._LINE_INDEX_MAP[source_qubit]
                source_qubit_state = self._STATE_CACHE_HASH_MAP[source_qubit_hash]
                target_qubit_hash = self._LINE_INDEX_MAP[target_qubit]
                target_qubit_state = self._STATE_CACHE_HASH_MAP[target_qubit_hash]
                print("I am here", source_qubit_state)
                cy_current_state = matmul(INV_CY, reduce(kron, [source_qubit_state, target_qubit_state]))
                nkp_state = nearest_kron_prod(cy_current_state)
                
                print(f"nkp = {nkp_state}")
                if nkp_state['ent']:
                    new_hash_key = uuid.uuid4().hex
                    self._LINE_INDEX_MAP[source_qubit] = new_hash_key
                    self._LINE_INDEX_MAP[source_qubit] = nkp_state['qubit-pair']
                    self._LINE_INDEX_MAP[target_qubit] = new_hash_key                
                    self._LINE_INDEX_MAP[target_qubit] = nkp_state['qubit-pair']

                if not nkp_state['ent']:
                    self._LINE_INDEX_MAP[source_qubit] = nkp_state['qubit-pair'][0]           
                    self._LINE_INDEX_MAP[target_qubit] = nkp_state['qubit-pair'][1]

        else:
            raise Exception("Exception TBA")


    
        
                                    

In [210]:
qc = QuantumCircuit(2)

In [211]:
qc.init()

In [212]:
qc.state_vector()

array([[1.],
       [0.],
       [0.],
       [0.]])

In [213]:
#qc.H(0)

In [214]:
qc.state_vector()

array([[1.],
       [0.],
       [0.],
       [0.]])

In [215]:
qc.CZ(1, 0)

1
I am here [[1.]
 [0.]]
nkp = {'size': 2, 'frobenius-norm': 0.0, 'qubit-pair': (array([[1.],
       [0.]]), array([[1.],
       [0.]])), 'ent': False}


In [216]:
qc.state_vector()

array([[1.],
       [0.],
       [0.],
       [0.]])